In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext('local')
spark = SparkSession(sc)

23/04/18 21:57:15 WARN Utils: Your hostname, jakub-G3-3590 resolves to a loopback address: 127.0.1.1; using 192.168.0.102 instead (on interface enp3s0)
23/04/18 21:57:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/18 21:57:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Wykorzystaj dane z bazy 'svrsqldev1.database.windows.net'
||
|--|
|SalesLT.Customer|
|SalesLT.ProductModel|
|SalesLT.vProductModelCatalogDescription|
|SalesLT.ProductDescription|
|SalesLT.Product|
|SalesLT.ProductModelProductDescription|
|SalesLT.vProductAndDescription|
|SalesLT.ProductCategory|
|SalesLT.vGetAllCategories|
|SalesLT.Address|
|SalesLT.CustomerAddress|
|SalesLT.SalesOrderDetail|
|SalesLT.SalesOrderHeader|

1. Po bierz wszystkie tabele z schematu SalesLt i zapisz lokalnie bez modyfikacji w formacie delta

Dla Chętnych
 Uzycie Nulls, fill, drop, replace
 * W kilku z tabel sprawdź ile jest nulls w kolumnach
 * Użyj funkcji fill żeby dodać wartości nie występujące w kolumnach dla wybranych tabel z null
 * Użyj funkcji drop żeby usunąć nulle.
 * Jakie są 3 najlepiej się sprzedające produkty?
 * Skąd są kupujący, który kupili te najlepsze produkty

# Zadanie 1
##  Product tables

In [3]:
product_path = "../../Lab_6/Product.csv"
product_description_path = "../../Lab_6/ProductDescription.csv"
product_model_path = "../../Lab_6/ProductModel.csv"
product_model_description_path = "../../Lab_6/ProductModelProductDescription.csv"

### Loading Data

In [48]:
df_product = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(product_path)
df_description = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(product_description_path)
df_model = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(product_model_path)
df_model_description = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(product_model_description_path)

### Joining DataFrames

In [49]:
df_join_inner = df_product.join(df_model).join(df_model_description).join(df_description)

In [50]:
df_join_left = df_product.join(df_model, how="left").join(df_model_description, how="left").join(df_description, how="left")

In [51]:
df_join_right = df_product.join(df_model, how="right").join(df_model_description, how="right").join(df_description, how="right")

In [52]:
df_join_outer = df_product.join(df_model, how="outer").join(df_model_description, how="outer").join(df_description, how="outer")

In [53]:
df_join_left_semi = df_product.join(df_model, how="left_semi").join(df_model_description, how="left_semi").join(df_description, how="left_semi")

In [54]:
df_join_left_anti = df_product.join(df_model, how="left_anti").join(df_model_description, how="left_anti").join(df_description, how="left_anti")

In [55]:
dataframes = [df_join_inner, df_join_left, df_join_right, df_join_outer, df_join_left_semi, df_join_left_anti]

### Explaining

In [37]:
for df in dataframes:
    df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastNestedLoopJoin BuildRight, Inner
   :- BroadcastNestedLoopJoin BuildRight, Inner
   :  :- BroadcastNestedLoopJoin BuildRight, Inner
   :  :  :- FileScan csv [ProductID#52,Name#53,ProductNumber#54,Color#55,StandardCost#56,ListPrice#57,Size#58,Weight#59,ProductCategoryID#60,ProductModelID#61,SellStartDate#62,SellEndDate#63,DiscontinuedDate#64,ThumbNailPhoto#65,ThumbnailPhotoFileName#66,rowguid#67,ModifiedDate#68] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/jakub/Studies/bigdata/Lab_6/Product.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<ProductID:int,Name:string,ProductNumber:string,Color:string,StandardCost:double,ListPrice:...
   :  :  +- BroadcastExchange IdentityBroadcastMode, [plan_id=801]
   :  :     +- FileScan csv [ProductModelID#128,Name#129,CatalogDescription#130,rowguid#131,ModifiedDate#132] Batched: false, DataFilters: [], Format: CSV, Loca

### Showing data

In [38]:
for df in dataframes:
    df.show()

+---------+--------------------+-------------+-----+------------+---------+----+-------+-----------------+--------------+-------------------+-----------+----------------+--------------------+----------------------+--------------------+--------------------+--------------+------------+------------------+--------------------+-------------------+--------------+--------------------+-------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|ProductID|                Name|ProductNumber|Color|StandardCost|ListPrice|Size| Weight|ProductCategoryID|ProductModelID|      SellStartDate|SellEndDate|DiscontinuedDate|      ThumbNailPhoto|ThumbnailPhotoFileName|             rowguid|        ModifiedDate|ProductModelID|        Name|CatalogDescription|             rowguid|       ModifiedDate|ProductModelID|ProductDescriptionID|Culture|             rowguid|       ModifiedDate|ProductDescriptionID|         Description|             ro

+---------+--------------------+-------------+-----+------------+---------+----+-------+-----------------+--------------+-------------------+-----------+----------------+--------------------+----------------------+--------------------+--------------------+--------------+------------+------------------+--------------------+-------------------+--------------+--------------------+-------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|ProductID|                Name|ProductNumber|Color|StandardCost|ListPrice|Size| Weight|ProductCategoryID|ProductModelID|      SellStartDate|SellEndDate|DiscontinuedDate|      ThumbNailPhoto|ThumbnailPhotoFileName|             rowguid|        ModifiedDate|ProductModelID|        Name|CatalogDescription|             rowguid|       ModifiedDate|ProductModelID|ProductDescriptionID|Culture|             rowguid|       ModifiedDate|ProductDescriptionID|         Description|             ro

+---------+--------------------+-------------+-----+------------+---------+----+-------+-----------------+--------------+-------------------+-----------+----------------+--------------------+----------------------+--------------------+--------------------+--------------+------------+------------------+--------------------+-------------------+--------------+--------------------+-------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|ProductID|                Name|ProductNumber|Color|StandardCost|ListPrice|Size| Weight|ProductCategoryID|ProductModelID|      SellStartDate|SellEndDate|DiscontinuedDate|      ThumbNailPhoto|ThumbnailPhotoFileName|             rowguid|        ModifiedDate|ProductModelID|        Name|CatalogDescription|             rowguid|       ModifiedDate|ProductModelID|ProductDescriptionID|Culture|             rowguid|       ModifiedDate|ProductDescriptionID|         Description|             ro

### Cleaning up

In [56]:
del df_product
del df_model
del df_description
del df_model_description

del df_join_left
del df_join_left_anti
del df_join_outer
del df_join_right
del df_join_inner
del df_join_left_semi

del dataframes

## Customer tables

In [4]:
address_path = "../../Lab_6/Address.csv"
customer_address_path = "../../Lab_6/CustomerAddress.csv"
customer_path = "../../Lab_6/Customer.csv"

### Loading data

In [58]:
df_address = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(address_path)
df_customer_address = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(customer_address_path)
df_customer = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(customer_path)

### Joining

In [61]:
df_join_inner = df_address.join(df_customer_address).join(df_customer)
df_join_left = df_address.join(df_customer_address, how="left").join(df_customer, how="left")
df_join_right = df_address.join(df_customer_address, how="right").join(df_customer, how="right")
df_join_outer = df_address.join(df_customer_address, how="outer").join(df_customer, how="outer")
df_join_left_semi = df_address.join(df_customer_address, how="left_semi").join(df_customer, how="left_semi")
df_join_left_anti = df_address.join(df_customer_address, how="left_anti").join(df_customer, how="left_anti")
dataframes = [df_join_inner, df_join_left, df_join_right, df_join_outer, df_join_left_semi, df_join_left_anti]

### Explaining

In [62]:
for df in dataframes:
    df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastNestedLoopJoin BuildRight, Inner
   :- BroadcastNestedLoopJoin BuildRight, Inner
   :  :- FileScan csv [AddressID#3865,AddressLine1#3866,AddressLine2#3867,City#3868,StateProvince#3869,CountryRegion#3870,PostalCode#3871,rowguid#3872,ModifiedDate#3873] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/jakub/Studies/bigdata/Lab_6/Address.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<AddressID:int,AddressLine1:string,AddressLine2:string,City:string,StateProvince:string,Cou...
   :  +- BroadcastExchange IdentityBroadcastMode, [plan_id=2287]
   :     +- FileScan csv [CustomerID#3900,AddressID#3901,AddressType#3902,rowguid#3903,ModifiedDate#3904] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/jakub/Studies/bigdata/Lab_6/CustomerAddress.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<CustomerID:int,A

### Showing data

In [63]:
for df in dataframes:
    df.show()

+---------+-----------------+------------+-------+-------------+-------------+----------+--------------------+-------------------+----------+---------+-----------+--------------------+-------------------+----------+---------+-----+-----------+----------+----------+------+--------------------+--------------------+--------------------+-------------------+--------------------+------------+--------------------+-------------------+
|AddressID|     AddressLine1|AddressLine2|   City|StateProvince|CountryRegion|PostalCode|             rowguid|       ModifiedDate|CustomerID|AddressID|AddressType|             rowguid|       ModifiedDate|CustomerID|NameStyle|Title|  FirstName|MiddleName|  LastName|Suffix|         CompanyName|         SalesPerson|        EmailAddress|              Phone|        PasswordHash|PasswordSalt|             rowguid|       ModifiedDate|
+---------+-----------------+------------+-------+-------------+-------------+----------+--------------------+-------------------+--------

### Cleaning

In [64]:
del df_address
del df_customer
del df_customer_address

del df_join_right
del df_join_left
del df_join_inner
del df_join_outer
del df_join_left_semi
del df_join_left_anti

del dataframes

## Sale order tables

In [6]:
detail_path = "../../Lab_6/SalesOrderDetail.csv"
header_path = "../../Lab_6/SalesOrderHeader.csv"

### Loading data

In [66]:
df_detail = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(detail_path)
df_header = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(header_path)

### Joining tables

In [69]:
df_join_inner = df_detail.join(df_header)
df_join_left = df_detail.join(df_header, how="left")
df_join_right = df_detail.join(df_header, how="right")
df_join_outer = df_detail.join(df_header, how="outer")
df_join_left_semi = df_detail.join(df_header, how="left_semi")
df_join_left_anti = df_detail.join(df_header, how="left_anti")
dataframes = [df_join_inner, df_join_left, df_join_right, df_join_outer, df_join_left_semi, df_join_left_anti]

### Explaining

In [70]:
for df in dataframes:
    df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastNestedLoopJoin BuildRight, Inner
   :- FileScan csv [SalesOrderID#4852,SalesOrderDetailID#4853,OrderQty#4854,ProductID#4855,UnitPrice#4856,UnitPriceDiscount#4857,LineTotal#4858,rowguid#4859,ModifiedDate#4860] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/jakub/Studies/bigdata/Lab_6/SalesOrderDetail.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<SalesOrderID:int,SalesOrderDetailID:int,OrderQty:int,ProductID:int,UnitPrice:double,UnitPr...
   +- BroadcastExchange IdentityBroadcastMode, [plan_id=3114]
      +- FileScan csv [SalesOrderID#4887,RevisionNumber#4888,OrderDate#4889,DueDate#4890,ShipDate#4891,Status#4892,OnlineOrderFlag#4893,SalesOrderNumber#4894,PurchaseOrderNumber#4895,AccountNumber#4896,CustomerID#4897,ShipToAddressID#4898,BillToAddressID#4899,ShipMethod#4900,CreditCardApprovalCode#4901,SubTotal#4902,TaxAmt#4903,Freight#4904,TotalDue#4905,Comme

### Showing data

In [71]:
for df in dataframes:
    df.show()

+------------+------------------+--------+---------+---------+-----------------+---------+--------------------+-------------------+------------+--------------+-------------------+-------------------+-------------------+------+---------------+----------------+-------------------+--------------+----------+---------------+---------------+-----------------+----------------------+-----------+---------+---------+----------+-------+--------------------+-------------------+
|SalesOrderID|SalesOrderDetailID|OrderQty|ProductID|UnitPrice|UnitPriceDiscount|LineTotal|             rowguid|       ModifiedDate|SalesOrderID|RevisionNumber|          OrderDate|            DueDate|           ShipDate|Status|OnlineOrderFlag|SalesOrderNumber|PurchaseOrderNumber| AccountNumber|CustomerID|ShipToAddressID|BillToAddressID|       ShipMethod|CreditCardApprovalCode|   SubTotal|   TaxAmt|  Freight|  TotalDue|Comment|             rowguid|       ModifiedDate|
+------------+------------------+--------+---------+------

### Cleaning

In [72]:
del df_detail
del df_header

del df_join_left_anti
del df_join_left
del df_join_left_semi
del df_join_outer
del df_join_inner
del df_join_right

del dataframes

# Zadanie 2

### Loading data

In [7]:
df_detail = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(detail_path)
df_header = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(header_path)
df_address = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(address_path)
df_customer_address = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(customer_address_path)
df_customer = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(customer_path)
df_product = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(product_path)
df_description = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(product_description_path)
df_model = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(product_model_path)
df_model_description = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(product_model_description_path)

In [8]:
print(df_product.columns)
print(df_model.columns)

['ProductID', 'Name', 'ProductNumber', 'Color', 'StandardCost', 'ListPrice', 'Size', 'Weight', 'ProductCategoryID', 'ProductModelID', 'SellStartDate', 'SellEndDate', 'DiscontinuedDate', 'ThumbNailPhoto', 'ThumbnailPhotoFileName', 'rowguid', 'ModifiedDate']
['ProductModelID', 'Name', 'CatalogDescription', 'rowguid', 'ModifiedDate']


In [10]:
sales_join_expr = df_detail["SalesOrderID"] == df_detail["SalesOrderID"]
customers_join_expr = df_address["AddressID"] == df_customer_address["AddressID"]
products_join_expr = df_product["ProductModelID"] == df_model["ProductModelID"]

23/04/18 21:59:34 WARN Column: Constructing trivially true equals predicate, 'SalesOrderID#17 = SalesOrderID#17'. Perhaps you need to use aliases.


## Joining Tables with duplicate column drop

In [13]:
df_sales = df_detail.join(df_header, sales_join_expr).drop(df_detail["SalesOrderID"])
df_customers = df_address.join(df_customer_address, customers_join_expr).drop(df_address["AddressID"])
df_products = df_product.join(df_model, products_join_expr).drop(df_product["ProductModelID"])

In [14]:
df_sales.columns

['SalesOrderDetailID',
 'OrderQty',
 'ProductID',
 'UnitPrice',
 'UnitPriceDiscount',
 'LineTotal',
 'rowguid',
 'ModifiedDate',
 'SalesOrderID',
 'RevisionNumber',
 'OrderDate',
 'DueDate',
 'ShipDate',
 'Status',
 'OnlineOrderFlag',
 'SalesOrderNumber',
 'PurchaseOrderNumber',
 'AccountNumber',
 'CustomerID',
 'ShipToAddressID',
 'BillToAddressID',
 'ShipMethod',
 'CreditCardApprovalCode',
 'SubTotal',
 'TaxAmt',
 'Freight',
 'TotalDue',
 'Comment',
 'rowguid',
 'ModifiedDate']

In [15]:
spark.stop()